In [2]:
!pip install transformers datasets torch sentence-transformers faiss-cpu

^C


     ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.4 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/44.4 kB 262.6 kB/s eta 0:00:01
     --------------------------------------- 44.4/44.4 kB 80.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:-

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\lenovo\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\transformers\\models\\deprecated\\efficientformer\\modeling_efficientformer.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached datasets-3.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached faiss_cpu-1.10.0-cp311-cp311-win_amd64.whl.metadata (4.5 kB)
  Using cached huggingface_hub-0.28.1-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached pyarrow-19.0.0-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.11.12-cp311-cp311-win_amd64.whl.metadata (8.0 kB)
  Using cached aiohappyeyeballs-2.4.6-py3-none-any.whl.metadata (

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.10 requires opencv-python>=4.6.0, which is not installed.
tsgm 0.0.7 requires tensorflow<2.16, but you have tensorflow 2.18.0 which is incompatible.

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# prompt: load a phi3 model from huggingface

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load Phi-3 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", torch_dtype=torch.float16, device_map="auto")


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

#load sentence embedding model 
retriever_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Example knowledge base 
docs = [
    "RAG is a technique that combines retrieval with generation.",
    "Phi-3 is a lightweight model for efficient text generation.",
    "FAISS is used for fast similarity search in large datasets."
]

# Convert documents to embeddings 
doc_embeddings = retriever_model.encode(docs)

# Create a FAISS index 
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [ ]:
def retrieve_documents(query, k=2):
    """Retrieve top-k most relevant documents."""
    query_embedding = retriever_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    return [docs[i] for i in indices[0]]

def generate_response(query):
    """Retrieve documents and generate a response using Phi-3."""
    retrieved_docs = retrieve_documents(query)
    context = "\n".join(retrieved_docs)

    # Format input prompt
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example Usage
query = "What is RAG and how does it work?"
response = generate_response(query)
print(response)
